In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My Drive/Kitti_Image/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/Kitti_Image


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import tensorflow as tf
import numpy as np
import pandas as pd

from preprocess import Preprocessor
from yolov3 import YoloV3
from postprocess import Postprocessor

num_classes = 3
class_names= ['Car','Cyclist','Pedestrian']
dest_path = "/content/gdrive/My Drive/Kitti_Image/input/detection-results"

**Train YOLO**

In [ ]:
!python train.py
#python train.py --checkpoint '/content/gdrive/My Drive/kitti/models/model-v1.0.1-epoch-10-loss-0.0065.tf'

Trained batch: 11 batch loss: 66.8591232 batch xy loss 10.3867378 batch wh loss 10.9088926 batch obj loss 41.1126518 batch_class_loss 4.45083714 epoch total loss: 1167.50769
Trained batch: 12 batch loss: 54.3849182 batch xy loss 8.64699459 batch wh loss 7.77132797 batch obj loss 34.9787102 batch_class_loss 2.98788333 epoch total loss: 1221.89258
Trained batch: 13 batch loss: 151.679108 batch xy loss 9.19206238 batch wh loss 104.791023 batch obj loss 33.6468811 batch_class_loss 4.04912281 epoch total loss: 1373.57166
Trained batch: 14 batch loss: 61.9763641 batch xy loss 9.14313412 batch wh loss 16.0870476 batch obj loss 30.4811859 batch_class_loss 6.26499271 epoch total loss: 1435.54797
Trained batch: 15 batch loss: 57.6565399 batch xy loss 9.05611897 batch wh loss 11.3672 batch obj loss 33.9155464 batch_class_loss 3.31767368 epoch total loss: 1493.20447
Trained batch: 16 batch loss: 69.4134445 batch xy loss 8.5512743 batch wh loss 19.2202339 batch obj loss 32.3824 batch_class_loss 9.2

**Validation**

---



In [ ]:
model = YoloV3(shape=(416, 416, 5), num_classes=num_classes, training=False)
model.load_weights('/content/gdrive/My Drive/kitti/models/model-v1.0.1-epoch-10-loss-20.3276.tf')
postprocess = Postprocessor(iou_thresh=0.5, score_thresh=0.2)

In [ ]:
files_list=['/content/gdrive/My Drive/kitti/tfrecords/val3.tfrecords',
            '/content/gdrive/My Drive/kitti/tfrecords/val7.tfrecords',
            '/content/gdrive/My Drive/kitti/tfrecords/val10.tfrecords',
            '/content/gdrive/My Drive/kitti/tfrecords/val22.tfrecords',
            '/content/gdrive/My Drive/kitti/tfrecords/val26.tfrecords',
            '/content/gdrive/My Drive/kitti/tfrecords/val35.tfrecords',
            '/content/gdrive/My Drive/kitti/tfrecords/val44.tfrecords',
            '/content/gdrive/My Drive/kitti/tfrecords/val53.tfrecords',
            '/content/gdrive/My Drive/kitti/tfrecords/val62.tfrecords',
            '/content/gdrive/My Drive/kitti/tfrecords/val65.tfrecords',
            '/content/gdrive/My Drive/kitti/tfrecords/val71.tfrecords',
            '/content/gdrive/My Drive/kitti/tfrecords/val85.tfrecords',
            '/content/gdrive/My Drive/kitti/tfrecords/val91.tfrecords',
            '/content/gdrive/My Drive/kitti/tfrecords/val98.tfrecords']

In [ ]:
dims_list = ['/content/gdrive/My Drive/kitti/input/val_img_dims/val_image_dims0.csv',
             '/content/gdrive/My Drive/kitti/input/val_img_dims/val_image_dims1.csv',
             '/content/gdrive/My Drive/kitti/input/val_img_dims/val_image_dims2.csv',
             '/content/gdrive/My Drive/kitti/input/val_img_dims/val_image_dims3.csv',
             '/content/gdrive/My Drive/kitti/input/val_img_dims/val_image_dims4.csv',
             '/content/gdrive/My Drive/kitti/input/val_img_dims/val_image_dims5.csv',
             '/content/gdrive/My Drive/kitti/input/val_img_dims/val_image_dims6.csv',
             '/content/gdrive/My Drive/kitti/input/val_img_dims/val_image_dims7.csv',
             '/content/gdrive/My Drive/kitti/input/val_img_dims/val_image_dims8.csv',
             '/content/gdrive/My Drive/kitti/input/val_img_dims/val_image_dims9.csv',
             '/content/gdrive/My Drive/kitti/input/val_img_dims/val_image_dims10.csv',
             '/content/gdrive/My Drive/kitti/input/val_img_dims/val_image_dims11.csv',
             '/content/gdrive/My Drive/kitti/input/val_img_dims/val_image_dims12.csv',
             '/content/gdrive/My Drive/kitti/input/val_img_dims/val_image_dims13.csv'] 

In [ ]:
for tf_file, df_file in zip (files_list, dims_list):
  preprocess = Preprocessor('combined')
  dataset = tf.data.Dataset.list_files(tf_file)
  dataset = tf.data.TFRecordDataset(dataset)
  dataset = dataset.map(preprocess)
  df = pd.read_csv(df_file)
  for data,n in zip (dataset, range(len(df))):
    filename = str(df.loc[n]['filename'])
    w = df.loc[n]['width']
    h = df.loc[n]['height']
    df1 = pd.DataFrame(columns= ['class_name','Probability','xmin','ymin','xmax','ymax'])
    x,y_true = data
    x = tf.expand_dims(x,axis=0)
    y_pred = model(x,training =False)
    y_pred_nms = postprocess(y_pred)

    num_predictions = y_pred_nms[3][0][0].numpy()
    predictions.append(num_predictions)

    for i in range(0,num_predictions):
      box = y_pred_nms[0][0][i].numpy()
      score = y_pred_nms[1][0][i][0].numpy()
      class_prob = y_pred_nms[2][0][i].numpy()
      clob_prob = class_prob * score
      values, indices = tf.math.top_k(clob_prob, k=1)
      for index, prob in zip(indices.numpy(), values.numpy()):
        confidence = prob
        class_name = class_names[index]
      xmin = box[0]* w
      ymin = box[1]* h
      xmax = box[2]* w 
      ymax = box[3]* h
      df1.loc[len(df1)] = [class_name,confidence,xmin,ymin,xmax,ymax]
    #result.append(df1)
    df1.to_csv(dest_path+'/'+filename, header=None, index=None, sep=' ', mode='a')

In [ ]:
!python main.py

: cannot connect to X server 


Show result on image

In [ ]:
class_names= ['Car','Cyclist','Pedestrian']
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w']
x1 = x[:,:,:,0:3]
im = tf.cast((x1[0] + 1) * 127.5, tf.int32)

plt.rcParams['figure.figsize'] = (20,20)
fig,ax = plt.subplots(1)
ax.imshow(im)
h, w, d = im.shape

for i in range(0, num_predictions):
    box = y_pred_nms[0][0][i].numpy()
    score = y_pred_nms[1][0][i][0].numpy()
    class_prob = y_pred_nms[2][0][i].numpy()
    clob_prob = class_prob * score
    values, indices = tf.math.top_k(clob_prob, k=1)
    text= []
    for index, prob in zip(indices.numpy(), values.numpy()):
        text.append('{} {:.1f}%'.format(class_names[index], prob*100))
    text = ', '.join(text)
    xmin = box[0] * w
    ymin = box[1] * h
    width = (box[2] - box[0]) * w 
    height = (box[3] - box[1]) * h
    color = colors[i % 8]
    rect = patches.Rectangle((xmin,ymin),width,height,linewidth=2,edgecolor=color,facecolor='none')
    ax.add_patch(rect)
    ax.annotate(text, (xmin, ymin - 5), color=color, weight='bold', 
                fontsize=18)

plt.show()

In [ ]:
ituy = class_prob * score
ituy*100

array([21.150715  ,  0.0921298 ,  0.25406817], dtype=float32)

Test for bugs

In [ ]:
postprocess = Postprocessor(iou_thresh=0.5, score_thresh=0.0)
output=[]
for tensor in y_true:
  output.append(tf.split(tensor,[4,1,3],axis=-1))
y_true_1 = (output[0],output[1],output[2])
y_true_nms = postprocess(y_true_1)

num_predictions = y_true_nms[3][0][0].numpy()
print('Number of predictions: ', num_predictions)

Number of predictions:  100


In [ ]:
class_names=['Car','Cyclist','Pedestrian']
im = tf.reshape(x,[416,416,3])
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w']
#im = tf.cast((x[0] + 1) * 127.5, tf.int32)

plt.rcParams['figure.figsize'] = (20,20)
fig,ax = plt.subplots(1)
ax.imshow(im)
h, w, d = im.shape

for i in range(0, num_predictions):
    box = y_true_nms[0][0][i].numpy()
    score = y_true_nms[1][0][i][0].numpy()
    class_prob = y_true_nms[2][0][i].numpy()
    values, indices = tf.math.top_k(class_prob, k=3)
    text= []
    for index, prob in zip(indices.numpy(), values.numpy()):
        text.append('{} {:.1f}%'.format(class_names[index], prob*100))
    text = ', '.join(text)
    xmin = box[0] * w
    ymin = box[1] * h
    width = (box[2] - box[0]) * w
    height = (box[3] - box[1]) * h
    color = colors[i % 8]
    rect = patches.Rectangle((xmin,ymin),width,height,linewidth=2,edgecolor=color,facecolor='None')
    ax.add_patch(rect)
    #ax.annotate(text, (xmin, ymin - 5), color=color, weight='bold', 
                #fontsize=18)

plt.show()